#### XML literals

In [ ]:
import scala.xml._

import scala.xml._


In [ ]:
val doc = <html><head><title>Fred's Memoirs</title></head><body>test xml </body></html>

doc: scala.xml.Elem = <html><head><title>Fred's Memoirs</title></head><body>test xml </body></html>


In [ ]:
val elem = <a href="http://scala-lang.org">The <em>Scala</em> language</a>
println(elem.label)
println(elem.child) // node sequences

for (n <- elem.child) println(n)

a
ArrayBuffer(The , <em>Scala</em>,  language)
The 
<em>Scala</em>
 language
elem: scala.xml.Elem = <a href="http://scala-lang.org">The <em>Scala</em> language</a>


#### Attributes

In [ ]:
val elem = <a href="http://scala-lang.org">The Scala language</a>
var meta = elem.attributes 

elem: scala.xml.Elem = <a href="http://scala-lang.org">The Scala language</a>
meta: scala.xml.MetaData =  href="http://scala-lang.org"


`attributes` yields an object of type MetaData，like a Map from keys to values

use `()` operator to access the value , yields a node sequence

In [ ]:
val url = elem.attributes("href")

val image = <img alt="San Jos&eacute; State University Logo" src="http://www.sjsu.edu/publicaffairs/pics/sjsu_logo_color_web.jpg"/>
val alt = image.attributes("alt") 

url: Seq[scala.xml.Node] = http://scala-lang.org
image: scala.xml.Elem = <img alt="San Jos&eacute; State University Logo" src="http://www.sjsu.edu/publicaffairs/pics/sjsu_logo_color_web.jpg"/>
alt: Seq[scala.xml.Node] = ArrayBuffer(San Jos, &eacute;,  State University Logo)


turn node sequence into a string by `.text`

In [ ]:
alt.text

res36: String = San Jos&eacute; State University Logo


San Jos&eacute; State University Logo

turn node sequence into a map by `asAttrMap`

In [ ]:
meta.asAttrMap

res38: Map[String,String] = Map(href -> http://scala-lang.org)


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

#### Embedded Expressions

In [ ]:
<ul>{for (i <- 1 to 5) yield <li>option {i}</li>}</ul> //scala block inside a scala block

res51: scala.xml.Elem = <ul><li>option 1</li><li>option 2</li><li>option 3</li><li>option 4</li><li>option 5</li></ul>


option 1 option 2 option 3 option 4 option 5

In [ ]:
//producing braces
<h3>The Natural Numbers {{1, 2, 3, ...}}</h3>

res55: scala.xml.Elem = <h3>The Natural Numbers {1, 2, 3, ...}</h3>


The Natural Numbers {1, 2, 3, ...}

compute attribute with expression

In [ ]:
<a att1={if (true) "desc1" else "desc"} href="#">empty link</a>

res61: scala.xml.Elem = <a att1="desc1" href="#">empty link</a>


empty link

#### CDATA

In [ ]:
val js = <script><![CDATA[ alert("Cold!")]]></script>

val g1 = <xml:group><li>Item 1</li><li>Item 2</li></xml:group>
val g2 = Group(Seq(<li>Item 1</li>, <li>Item 2</li>))

// iterate
val items = <li>Item 1</li><li>Item 2</li>
for (n <- <xml:group>{items}</xml:group>) yield n
// Yields two li elements
for (n <- <ol>{items}</ol>) yield n
// Yields one ol element

js: scala.xml.Elem = <script> alert(&quot;Cold!&quot;)</script>
g1: scala.xml.Group = <li>Item 1</li><li>Item 2</li>
g2: scala.xml.Group = <li>Item 1</li><li>Item 2</li>
items: scala.xml.NodeBuffer = ArrayBuffer(<li>Item 1</li>, <li>Item 2</li>)
res68: scala.xml.NodeSeq = NodeSeq(<ol><li>Item 1</li><li>Item 2</li></ol>)


Item 1 Item 2

#### XPath-like

> use `\` ,`\\` replace `/` ,`//` (`//` denotes comments)

In [ ]:
val list = <dl><dt>Java</dt><dd>Gosling</dd><dt>Scala</dt><dd>Odersky</dd></dl>
val languages = list \ "dt"

list: scala.xml.Elem = <dl><dt>Java</dt><dd>Gosling</dd><dt>Scala</dt><dd>Odersky</dd></dl>
languages: scala.xml.NodeSeq = NodeSeq(<dt>Java</dt>, <dt>Scala</dt>)


`_` wildcard, matches any element 

In [ ]:
val long = <html><body><ul data="ul">{for (i <- 1 to 2) yield <li>a{i}</li>}</ul><ol data="ol">{for (i <- 1 to 2) yield <li>b{i}</li>}</ol></body></html>
long \ "html" \ "_" \ "li"  // matches nothing
long \ "body" \ "_" \ "li" //matches all li

long: scala.xml.Elem = <html><body><ul data="ul"><li>a1</li><li>a2</li></ul><ol data="ol"><li>b1</li><li>b2</li></ol></body></html>
res4: scala.xml.NodeSeq = NodeSeq(<li>a1</li>, <li>a2</li>, <li>b1</li>, <li>b2</li>)


a1 a2 b1 b2

`\\` locates descendants at any depth

In [ ]:
long \\ "li"

res90: scala.xml.NodeSeq = NodeSeq(<li>a1</li>, <li>a2</li>, <li>b1</li>, <li>b2</li>)


a1 a2 b1 b2

`@` locates attributes   (no wildcard for this)

In [ ]:
long \\ "@data"

res96: scala.xml.NodeSeq = NodeSeq(ul, ol)


ulol

#### Pattern Matching

In [ ]:
val m  = <li><img alt="image"/><a href="#">test</a></li>

m match {
//   case <li>{_}</li> => println(1)  // match one element
//   case <li>{_*}</li> => println(2) // match multi element
  case <li>{children @ _*}</li> => { // bind a variable
    println(children)
    for(c <- children) println(c)
  }
  case _ => None
}

ArrayBuffer(<img alt="image"/>, <a href="#">test</a>)
<img alt="image"/>
<a href="#">test</a>
m: scala.xml.Elem = <li><img alt="image"/><a href="#">test</a></li>
res22: Any = ()


()

**Note:** `case` clause should have only one element and can't have attributes

#### Modifying

before edit, have to create a copy

`copy(label, attributes, child, prefix, scope)`  paramaters are _named parameters_，specify changed paramters

In [ ]:
val list = <ul><li>Fred</li><li>Wilma</li></ul>
val list2 = list.copy(label = "ol")

list: scala.xml.Elem = <ul><li>Fred</li><li>Wilma</li></ul>
list2: scala.xml.Elem = <ol><li>Fred</li><li>Wilma</li></ol>


In [ ]:
list.copy(child = list.child ++ <li> add a child</li>)

res27: scala.xml.Elem = <ul><li>Fred</li><li>Wilma</li><li> add a child</li></ul>


Fred Wilma add a child

In [ ]:
val image = <img src="hamster.jpg"/>
val image2 = image % Attribute(null, "alt", "add an attribute", Null)
val image3 = image % Attribute(null, "alt", "add more then one attribute",Attribute(null, "src", "updateAttribute.jpg", Null))

image: scala.xml.Elem = <img src="hamster.jpg"/>
image2: scala.xml.Elem = <img alt="add an attribute" src="hamster.jpg"/>
image3: scala.xml.Elem = <img src="updateAttribute.jpg" alt="add more then one attribute"/>


#### Transforming

In [ ]:
import scala.xml.transform._
val rule1 = new RewriteRule {
  override def transform(n: Node) = n match {
    case e @ <ul>{_*}</ul> => e.asInstanceOf[Elem].copy(label = "ol")
    case _ => n
  }
}
val transformed = new RuleTransformer(rule1).transform(list)
// can apply any number of rules
// val transformer = new RuleTransformer(rule1, rule2, rule3);

import scala.xml.transform._
rule1: scala.xml.transform.RewriteRule = <function1>
transformed: Seq[scala.xml.Node] = List(<ol><li>Fred</li><li>Wilma</li></ol>)
